In [1]:
#paquetes 
import numpy as np
import networkx as nx
import matplotlib.pylab as plt
%matplotlib inline
import os
#from random import shuffle
import math
import pandas as pd
from scipy import optimize


PARA CADA RED: Me quedo con la componente gigante y me saco de encima el resto. Primero, voy a calcular, para todos los nodos de todas las redes, los siguientes parámetros: 

-degree centrality https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.centrality.html 

-eigenvector centrality https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.eigenvector_centrality.html#networkx.algorithms.centrality.eigenvector_centrality 

-subgraph centrality https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.communicability_centrality.html#networkx.algorithms.centrality.communicability_centrality 

-shortest-path centrality https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.betweenness_centrality.html#networkx.algorithms.centrality.betweenness_centrality 

-current flow betweeness centrality https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.current_flow_betweenness_centrality.html#networkx.algorithms.centrality.current_flow_betweenness_centrality 

Ordeno mis nodos de mayor a menor para cada parametro. Voy sacando de a un nodo y me fijo de que tamaño queda el hub más grande. Eje x es fracción de nodos sacados, y eje y es la fracción de nodos en el hub más grande. y=nodos en el componene gigante/tamaño de componente gigante original

Después, para la tabla 3, veo el efecto de sacar todos (nro=E) mis nodos esenciales. Además, veo el efecto de sacar E nodos no esenciales (osea la misma cantidad que esenciales haya en la red) de manera random, pero respetando el grado. Por ejemplo, si tengo 10 esenciales de grado 8 y 16 esenciales de grado 7, tengo que sacar 10 noesenciales de grado 8 y 16 noesenciales de grado 7 etc... Eso se hace muchass veces de manera random y se calcula el valor medio y dispersion..

Comentario: En realidad, lo formalmente bien seria ir redefiniendo todos mis parametros en cada iteración, pero como tardaria mil años, entonces lo hago "cada tanto". Saco ponele los 10 mas grandes y recalculo parametros en mi nueva componente gigante, etc etc Despues puedo comparar si realmente cambia o no, y cuánto.

# RED AP-MS

# Me quedo con la componente gigante

In [2]:
def ldata(archive):
    f=open(archive)
    data=[]
    for line in f:
        line=line.strip()
        col=line.split()
        data.append(col)
    return data

def grafo(datosRed):
    G = nx.Graph()
    for i in range(np.shape(datosRed)[0]):
        G.add_edges_from([(datosRed[i][0],datosRed[i][1])])
    return G 

In [3]:
APMS = ldata('yeast_AP-MS.txt')
GAPMS = grafo(APMS)
# Y2H=ldata('yeast_Y2H.txt')
# GY2H=grafo(Y2H)

# Calculo coeficientes de centralidad

In [5]:
# degrees=dict(Gc.degree())
# eigenvectors=nx.eigenvector_centrality_numpy(Gc)
#subgraph=sc = nx.communicability_betweenness_centrality(Gc)
#shortpath=nx.betweenness_centrality(Gc)
#currentflow=nx.current_flow_betweenness_centrality(Gc)

def param(G):
   #parametros=[dict(G.degree()),nx.subgraph_centrality(G),nx.betweenness_centrality(G)] #,nx.current_flow_betweenness_centrality(G)]  
    parametros=[nx.eigenvector_centrality_numpy(G)]
    return parametros

In [6]:
#FUNCIÓN QUE ORDENA LOS NODOS DE MANERA DECRECIENTE SEGÚN PARÁMETRO DEL 'CRITERIO'
def ordenanodos(criterio): #criterio=diccionario 'nombredenodo':valor de parametro
    parametro=criterio.values()
    parametroordenado=sorted(set(parametro),reverse=True)
    nodosordenados=[]
    for j in parametroordenado:
        for key,value in criterio.items():
            if value==j:
                nodosordenados.append(key)
    return(nodosordenados)

# DESARM

In [14]:
def desarm(G):

    FN=[]
    LGC=[]
    
    Gc = max(nx.connected_component_subgraphs(G), key=len)
    parametros=param(Gc)
    
    for k in range(len(parametros)):
        parametro=parametros[k]
              
        Pimp=ordenanodos(parametro)
        GcDesarmable = max(nx.connected_component_subgraphs(G), key=len) #Me creo una copia de la componente Gigante Original.
        LengthOriginalGc=len(Gc)
        LargestGc=[]
        
        for i in range(len(Gc)):
            GcDesarmable.remove_node(Pimp[0])
            LargestGc.append(float(len(max(nx.connected_component_subgraphs(GcDesarmable), key=len)))/LengthOriginalGc)
            GcDesarmable = max(nx.connected_component_subgraphs(GcDesarmable), key=len)
            
            parametro=param(GcDesarmable)[k]
            Pimp=ordenanodos(parametro)
            print('lenGc',len(GcDesarmable))
            if len(GcDesarmable)<5:
                it=i
                break
                
        FN.append(np.linspace(1,it+1,it+1)/LengthOriginalGc)
        LGC.append(LargestGc)
    
    return FN,LGC

In [15]:
FN,LGC=desarm(GAPMS)

lenGc 1003
lenGc 1002
lenGc 1001
lenGc 1000
lenGc 999
lenGc 998
lenGc 997
lenGc 996
lenGc 995
lenGc 994
lenGc 993
lenGc 989
lenGc 987
lenGc 986
lenGc 979
lenGc 977
lenGc 976
lenGc 975
lenGc 974
lenGc 973
lenGc 972
lenGc 971
lenGc 970
lenGc 969
lenGc 965
lenGc 963
lenGc 962
lenGc 961
lenGc 960
lenGc 959
lenGc 958
lenGc 957
lenGc 956
lenGc 955
lenGc 953
lenGc 951
lenGc 950
lenGc 949
lenGc 948
lenGc 947
lenGc 945
lenGc 944
lenGc 943
lenGc 942
lenGc 941
lenGc 940
lenGc 939
lenGc 938
lenGc 937
lenGc 936
lenGc 935
lenGc 934
lenGc 933
lenGc 930
lenGc 929
lenGc 928
lenGc 927
lenGc 926
lenGc 925
lenGc 924
lenGc 923
lenGc 922
lenGc 893
lenGc 892
lenGc 891
lenGc 890
lenGc 889
lenGc 888
lenGc 887
lenGc 886
lenGc 885
lenGc 884
lenGc 883
lenGc 882
lenGc 881
lenGc 880
lenGc 879
lenGc 878
lenGc 877
lenGc 876
lenGc 875
lenGc 874
lenGc 873
lenGc 872
lenGc 871
lenGc 870
lenGc 869
lenGc 868
lenGc 867
lenGc 866
lenGc 865
lenGc 864
lenGc 863
lenGc 862
lenGc 861
lenGc 860
lenGc 859
lenGc 858
lenGc 857
lenGc 

In [ ]:
# FN,LGC=desarm_degree(Gc)

plt.plot(FN[0],LGC[0],label='shortest path')
# plt.plot(FN[1],LGC[1],label='subgraph')
# plt.plot(FN[2],LGC[2],label='subgraph')
# plt.plot(FN[3],LGC[3],label='current_flow')
plt.xlabel('Fraction of nodes')
plt.ylabel('Largest Giant Component')
plt.legend()

# Vamos ahora con EIGENVECTORS

In [ ]:
eigenvectors=nx.eigenvector_centrality(Gc)
EigenvectorImp=ordenanodos(eigenvectors)

In [ ]:
GcEigen = max(nx.connected_component_subgraphs(GAPMS), key=len)
LengthOriginalGc=len(Gc)
FractionEigen=np.linspace(1,650,650)/LengthOriginalGc
LargestGcEigen=[]
for nodo in EigenvectorImp[0:650]:
    GcEigen.remove_node(nodo)
    LargestGcEigen.append(float(len(max(nx.connected_component_subgraphs(GcEigen), key=len)))/LengthOriginalGc)


In [ ]:
n=120  #600/m
m=5  #CADA CUANTO RECALCULO
GcEigen2 = max(nx.connected_component_subgraphs(GAPMS), key=len)
LengthOriginalGc=len(Gc)
FractionEigen2=np.linspace(1,n*m,n*m)/LengthOriginalGc
LargestGcEigen2=[]
for i in range(n):
    eigenvectors2=nx.eigenvector_centrality_numpy(GcEigen2)
    EigenvectorImp2=ordenanodos(eigenvectors2)
    for j in range(m):
        GcEigen2.remove_node(EigenvectorImp2[j])
        LargestGcEigen2.append(float(len(max(nx.connected_component_subgraphs(GcEigen2), key=len)))/LengthOriginalGc)


In [ ]:
plt.plot(FractionEigen2,LargestGcEigen2,label='recalculando cada 5') #en azul
plt.plot(FractionEigen,LargestGcEigen,label='sin recalcular') #en naranja
plt.xlabel('Fraction of nodes')
plt.ylabel('Largest Giant Component')
plt.legend()

# Vamos ahora con SHORTEST PATH

In [ ]:
shortpath=nx.betweenness_centrality(Gc)
ShortpathImp=ordenanodos(shortpath)

GcShort = max(nx.connected_component_subgraphs(GAPMS), key=len)
LengthOriginalGc=len(Gc)
FractionShort=np.linspace(1,650,650)/LengthOriginalGc
LargestGcShort=[]
for nodo in ShortpathImp[0:650]:
    GcShort.remove_node(nodo)
    LargestGcShort.append(float(len(max(nx.connected_component_subgraphs(GcShort), key=len)))/LengthOriginalGc)


In [ ]:
n=120  #600/m
m=5  #CADA CUANTO RECALCULO
GcShort2 = max(nx.connected_component_subgraphs(GAPMS), key=len)
LengthOriginalGc=len(Gc)
FractionShort2=np.linspace(1,n*m,n*m)/LengthOriginalGc
LargestGcShort2=[]
for i in range(n):
    shortpath2=nx.betweenness_centrality(GcShort2)
    ShortImp2=ordenanodos(shortpath2)
    for j in range(m):
        GcShort2.remove_node(ShortImp2[j])
        LargestGcShort2.append(float(len(max(nx.connected_component_subgraphs(GcShort2), key=len)))/LengthOriginalGc)


In [ ]:
plt.plot(FractionShort2,LargestGcShort2,label='recalculando cada 5') #en azul
plt.plot(FractionShort,LargestGcShort,label='sin recalcular') #en naranja
plt.xlabel('Fraction of nodes')
plt.ylabel('Largest Giant Component')
plt.legend()

# Vamos ahora con CURRENT FLOW

In [ ]:
currentflow=nx.current_flow_betweenness_centrality(Gc)
CurrentflowImp=ordenanodos(currentflow)

GcCurrent = max(nx.connected_component_subgraphs(GAPMS), key=len)
LengthOriginalGc=len(Gc)
FractionCurrent=np.linspace(1,650,650)/LengthOriginalGc
LargestGcCurrent=[]
for nodo in CurrentflowImp[0:650]:
    GcCurrent.remove_node(nodo)
    LargestGcCurrent.append(float(len(max(nx.connected_component_subgraphs(GcCurrent), key=len)))/LengthOriginalGc)


In [ ]:
n=70  #600/m
m=5  #CADA CUANTO RECALCULO
GcCurrent2 = max(nx.connected_component_subgraphs(GAPMS), key=len)
LengthOriginalGc=len(Gc)
FractionCurrent2=np.linspace(1,n*m,n*m)/LengthOriginalGc
LargestGcCurrent2=[]
for i in range(n):
    currentflow2=nx.current_flow_betweenness_centrality(max(nx.connected_component_subgraphs(GcCurrent2),key=len))
    CurrentflowImp2=ordenanodos(currentflow2)
    for j in range(m):
        GcCurrent2.remove_node(CurrentflowImp2[j])
        LargestGcCurrent2.append(float(len(max(nx.connected_component_subgraphs(GcCurrent2), key=len)))/LengthOriginalGc)


In [ ]:
plt.plot(FractionCurrent2,LargestGcCurrent2,label='recalculando cada 5') #en azul
plt.plot(FractionCurrent,LargestGcCurrent,label='sin recalcular') #en naranja
plt.xlabel('Fraction of nodes')
plt.ylabel('Largest Giant Component')
plt.legend()

# por ultimo, SUBGRAPH

In [ ]:
subgraph=nx.subgraph_centrality(Gc)
SubgraphImp=ordenanodos(subgraph)

GcSubgraph = max(nx.connected_component_subgraphs(GAPMS), key=len)
LengthOriginalGc=len(Gc)
FractionSubgraph=np.linspace(1,650,650)/LengthOriginalGc
LargestGcSubgraph=[]
for nodo in SubgraphImp[0:650]:
    GcSubgraph.remove_node(nodo)
    LargestGcSubgraph.append(float(len(max(nx.connected_component_subgraphs(GcSubgraph), key=len)))/LengthOriginalGc)


In [ ]:
n=120  #600/m
m=5  #CADA CUANTO RECALCULO
GcSubgraph2 = max(nx.connected_component_subgraphs(GAPMS), key=len)
LengthOriginalGc=len(Gc)
FractionSubgraph2=np.linspace(1,n*m,n*m)/LengthOriginalGc
LargestGcSubgraph2=[]
for i in range(n):
    subgraph2=nx.nx.subgraph_centrality(GcSubgraph2)
    SubgraphImp2=ordenanodos(subgraph2)
    for j in range(m):
        GcSubgraph2.remove_node(SubgraphImp2[j])
        LargestGcSubgraph2.append(float(len(max(nx.connected_component_subgraphs(GcSubgraph2), key=len)))/LengthOriginalGc)


In [ ]:
plt.plot(FractionSubgraph2,LargestGcSubgraph2,label='recalculando cada 5') #en azul
plt.plot(FractionSubgraph,LargestGcSubgraph,label='sin recalcular') #en naranja
plt.xlabel('Fraction of nodes')
plt.ylabel('Largest Giant Component')
plt.legend()

# Me guardo todo en .txt

In [ ]:
DegreeAPMS=np.zeros((len(FractionNodes2),2))
DegreeAPMS[:,0]=FractionNodes2
DegreeAPMS[:,1]=LargestGc2

EigenAPMS=np.zeros((len(FractionEigen2),2))
EigenAPMS[:,0]=FractionEigen2
EigenAPMS[:,1]=LargestGcEigen2

ShortpathAPMS=np.zeros((len(FractionShort2),2))
ShortpathAPMS[:,0]=FractionShort2
ShortpathAPMS[:,1]=LargestGcShort2

CurrentflowAPMS=np.zeros((len(FractionCurrent2),2))
CurrentflowAPMS[:,0]=FractionCurrent2
CurrentflowAPMS[:,1]=LargestGcCurrent2



In [ ]:
SubgraphAPMS=np.zeros((len(FractionSubgraph2),2))
SubgraphAPMS[:,0]=FractionSubgraph2
SubgraphAPMS[:,1]=LargestGcSubgraph2

In [ ]:
#np.savetxt('DegreeAPMS.txt', DegreeAPMS, delimiter=',')
#np.savetxt('EigenAPMS.txt', EigenAPMS, delimiter=',')
#np.savetxt('ShortpathAPMS.txt', ShortpathAPMS, delimiter=',')
#np.savetxt('CurrentflowAPMS.txt', CurrentflowAPMS, delimiter=',') 
#np.savetxt('SubgraphAPMS.txt', SubgraphAPMS, delimiter=',') 


# Importo para graficar

In [ ]:
DegreeAPMS=np.loadtxt('DegreeAPMS.txt',delimiter=',')
EigenAPMS=np.loadtxt('EigenAPMS.txt',delimiter=',')
ShortpathAPMS=np.loadtxt('ShortpathAPMS.txt',delimiter=',')
CurrentflowAPMS=np.loadtxt('CurrentflowAPMS.txt',delimiter=',')
SubgraphAPMS=np.loadtxt('SubgraphAPMS.txt',delimiter=',')

In [ ]:
plt.figure(figsize=(12,9))
plt.plot(DegreeAPMS[:,0],DegreeAPMS[:,1],label='Degree') 
plt.plot(EigenAPMS[:,0],EigenAPMS[:,1],label='Eigenvector') 
plt.plot(ShortpathAPMS[:,0],ShortpathAPMS[:,1],label='Short-path') 
plt.plot(CurrentflowAPMS[:,0],CurrentflowAPMS[:,1],label='Current flow')
plt.plot(SubgraphAPMS[:,0],SubgraphAPMS[:,1],label='Subgraph')
plt.plot(FractionNodesEsenencial,LargestGcEsencial,'o', label='Essentials')


plt.xlabel('Fraction of nodes')
plt.ylabel('Largest Giant Component')
plt.legend()

# Queda pendiente...

Ir sacando de manera random los nodos

Sacar los que son esenciales

Descubrir qué comando encuentra la subgraph centrality

# Asignación de esencialidad a nodos

In [ ]:
essentiality=ldata('Essential.txt')

ess=[]
for i in range(len(essentiality)):
    ess.append(essentiality[i][1])
    
def essentiality(nodo): 
    if len({nodo}.intersection(set(ess)))==0:
        e=0
    else:
        e=1
    return e    

dict_ess = { nodo: essentiality(nodo) for nodo  in GAPMS.nodes() }

def AssignEssentiality(G,dict_ess):
    for n in G.nodes:
        G.nodes[n]["essentiality"] = dict_ess[n]
    return

AssignEssentiality(GAPMS,dict_ess)

In [ ]:
#Te devuelve una lista de los nodos esenciales de un grafo
def NodosEsenciales(G):
    nodosesenciales=[]
    for nodo in list(set(G.nodes())):
        if G.nodes.data("essentiality")[nodo]==1:
            nodosesenciales.append(nodo)
    return(nodosesenciales)
    

# Saco los nodos esenciales de mi componente gigante

In [ ]:
essentiality=ldata('Essential.txt')

ess=[]
for i in range(len(essentiality)):
    ess.append(essentiality[i][1])
    
def essentiality(nodo): 
    if len({nodo}.intersection(set(ess)))==0:
        e=0
    else:
        e=1
    return e   

def AssignEssentiality(G,dict_ess):
    for n in G.nodes:
        G.nodes[n]["essentiality"] = dict_ess[n]
    return

def NodosEsenciales(G):
    dict_ess = { nodo: essentiality(nodo) for nodo  in G.nodes() }
    AssignEssentiality(GAPMS,dict_ess)
    nodosesenciales=[]
    for nodo in list(set(G.nodes())):
        if G.nodes.data("essentiality")[nodo]==1:
            nodosesenciales.append(nodo)
    return(nodosesenciales)

def SinEsenciales(G)
    GcEsencial = max(nx.connected_component_subgraphs(G), key=len) #Me creo una copia de la componente Gigante Original.
    LengthOriginalGc=len(max(nx.connected_component_subgraphs(G), key=len))
    nodosesenciales=NodosEsenciales(GcEsencial)
    for nodo in nodosesenciales:
        GcEsencial.remove_node(nodo)
    FN=float(len(nodosesenciales))/LengthOriginalGc
    LGC=float(len(max(nx.connected_component_subgraphs(GcEsencial), key=len)))/LengthOriginalGc
    return FN,LGC

In [ ]:
FN_APMS,LGC_APMS=SinEsenciales(GAPMS)
FN_Y2H,LGC_Y2H=SinEsenciales(GY2H)
FN_LIT,LGC_LIT=SinEsenciales(GLIT)
FN_LIT_R,LGC_LIT_r=SinEsenciales(GLIT_R)

In [ ]:
FractionNodesEsenencial=0.4043824701195219
LargestGcEsencial= 0.3237051792828685
